# CLIENT

многое не успел но это честная работа )))

Модельки взял ультрапростые 

сначала запускаем сервер : python3 server.py

## Hello World

In [19]:
import requests
response = requests.get("http://127.0.0.1:5000")
print(response.text)


Hello, World!


## Обучаем две разные модели и сохраняем их на сервере :

In [20]:
#Обработку некорректных данных сделать не успеваю (())

import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.linear_model import LinearRegression

X = np.random.rand(100, 3).tolist()  # Признаки
y = list(np.random.rand(100))  # Целевая переменная
data = {
    "X": X,
    "y": y,
    "config": {
        "model_name": "catboost_regressor228",
        "model_type": "CatboostRegressor"
    }
}
data2 = {
    "X": X,
    "y": y,
    "config": {
        "model_name": "sklearn_regressor222",
        "model_type": "sklearn.linear_model"
    }
}

response = requests.post("http://127.0.0.1:5000/fit" , json = data)
result = response.json()
print(result['train_log'])
print(result['message'])

response = requests.post("http://127.0.0.1:5000/fit" , json = data2)
result = response.json()
print(result['train_log'])
print(result['message'])

CatboostRegressor fitting :
Success
Model 'catboost_regressor228' trained and saved

Sklearn linear model fitting :
Success
Model 'sklearn_regressor222' trained and saved


## Обучаем модель у себя и загружаем ее на сервер, потом запускаем ее на сервере

In [21]:
import joblib
import pickle

X = np.random.rand(100, 3).tolist()  # Признаки
y = list(np.random.rand(100))  # Целевая переменная
X = np.array(X)
y = np.array(y)
train_data = Pool(data=X, label=y)
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6 )
model.fit(train_data)

X_test = np.random.rand(10, 3)
# print(model.predict(test))

data = {
    "decoded_model" : pickle.dumps(model).decode("latin1") ,
    "config": {
        "model_name": "catboost_regressor_trained_in_my_computer",
        "model_type": "CatboostRegressor"
    }
}
response = requests.post("http://127.0.0.1:5000/load" , json = data)
result = response.json()
print(result)


data = {
    "X_test" : X_test.tolist() ,
    "config": {
        "model_name": "catboost_regressor_trained_in_my_computer",
        "model_type": "CatboostRegressor"
    }
}
response = requests.post("http://127.0.0.1:5000/predict" , json = data)
result = response.json()
print(result)


0:	learn: 0.2938080	total: 1.06ms	remaining: 105ms
1:	learn: 0.2910002	total: 2.09ms	remaining: 102ms
2:	learn: 0.2884846	total: 3.16ms	remaining: 102ms
3:	learn: 0.2844897	total: 4.11ms	remaining: 98.7ms
4:	learn: 0.2817086	total: 5.43ms	remaining: 103ms
5:	learn: 0.2772053	total: 6.22ms	remaining: 97.4ms
6:	learn: 0.2744746	total: 8.32ms	remaining: 111ms
7:	learn: 0.2710062	total: 9.53ms	remaining: 110ms
8:	learn: 0.2678063	total: 10.9ms	remaining: 110ms
9:	learn: 0.2642616	total: 12.1ms	remaining: 109ms
10:	learn: 0.2618032	total: 12.9ms	remaining: 104ms
11:	learn: 0.2591770	total: 14.7ms	remaining: 107ms
12:	learn: 0.2566011	total: 16.1ms	remaining: 108ms
13:	learn: 0.2546390	total: 16.9ms	remaining: 104ms
14:	learn: 0.2526817	total: 17.5ms	remaining: 99.3ms
15:	learn: 0.2513375	total: 18.3ms	remaining: 95.8ms
16:	learn: 0.2498975	total: 19.2ms	remaining: 93.6ms
17:	learn: 0.2484229	total: 19.6ms	remaining: 89.3ms
18:	learn: 0.2469131	total: 20.3ms	remaining: 86.6ms
19:	learn: 0.24

## Выгружаем обученную модель с сервера, запускаем ее у себя

In [22]:
import pickle
data = {
    "config": {
        "model_name": "catboost_regressor_trained_in_my_computer",
        "model_type": "CatboostRegressor"
    }
}
response = requests.post("http://127.0.0.1:5000/unload" , json = data)
result = response.json()
model = pickle.loads(result['decoded_model'].encode('latin1'))
X_test = np.random.rand(10, 3)
print(model.predict(X_test))

[0.52361312 0.51564361 0.52089447 0.55691841 0.57126445 0.30340279
 0.56625367 0.52344467 0.59227509 0.6675322 ]


## Удаляем одну конкретную и все модели с сервера

In [23]:
data = {
    "config": {
        "model_name": "catboost_regressor_trained_in_my_computer",
        "model_type": "CatboostRegressor"
    }
}
response = requests.post("http://127.0.0.1:5000/remove" , json = data)
result = response.json()
print(result)

{'message': "Model 'catboost_regressor_trained_in_my_computer' removed"}


In [24]:
# Не придумал что в конфиге передать если мы хотим все удалить
data = {
    "config": {}
}
response = requests.post("http://127.0.0.1:5000/remove_all" , json = data)
result = response.json()
print(result)

{'message': 'All models removed'}
